In [17]:
from functools import partial
import random
import typing as tp

import torch
from torch import nn
from torch.nn import functional as F
import torchaudio as ta

from bm.models.common import (
    ConvSequence, ScaledEmbedding, SubjectLayers,
    DualPathRNN, ChannelMerger, ChannelDropout, pad_multiple
)
from bm.models import SimpleConv
from bm.models import CNNTransformer
from bm import play

In [18]:
simpleconv={
      "in_channels": {"meg" : 207},
      "out_channels" :1024,
      "hidden":
        {"meg": 320},
      "batch_norm": True,
      "depth": 4,
      "dilation_period": 1,
      "kernel_size": [3,3,3,3,3,3,3,3,3,3],
      "skip": True, 
      "subject_layers": False,
      "subject_dim": 0,
      "complex_out": False, 
      "glu": 2, 
      "glu_context": 1, 
      "merger": False,
      "initial_linear": 270,
      "avg_pool_out" : False,
      "adaptive_pooling_size" : 1,
      "strides" : [1,1,1,1,1,1,1,1,1,1],
      "padding": [1,1,1,1,1,1,1,1,1,1,1,1],
      "gelu": True,
      "merger_pos_dim": 2048,
      "auto_padding" : False,
      "flatten_out" : True,
       "conv_dropout" : 0.2,
     "is_deformable_conv": False,
       "seq_len" : 361, 
      "n_subjects" : 27,
      "flatten_out_channels" : 32}

In [19]:
in_channels = {"meg" : 207}
model_chout =270

In [20]:
model = SimpleConv(**simpleconv)

32


In [21]:
model

SimpleConv(
  (initial_linear): Sequential(
    (0): Conv1d(207, 270, kernel_size=(1,), stride=(1,))
  )
  (encoders): ModuleDict(
    (meg): ConvSequence(
      (sequence): ModuleList(
        (0): Sequential(
          (0): Conv1d(270, 320, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): GELU(approximate='none')
          (3): Dropout(p=0.2, inplace=False)
        )
        (1-2): 2 x Sequential(
          (0): Conv1d(320, 320, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): GELU(approximate='none')
          (3): Dropout(p=0.2, inplace=False)
        )
        (3): Sequential(
          (0): Conv1d(320, 32, kernel_size=(3,), stride=(1,), padding=(1,))
        )
      )
      (glus): ModuleList(
        (0): None
        (1): Sequential(
          (0): Conv1d(320, 640, ke

In [6]:
batch =torch.rand(size=(256, 207, 362))

In [7]:
class SegmentBatch():
    def __init__(self, indx, batch):
        self.subject_index =indx
        self.meg = batch

In [8]:
segment_batch =SegmentBatch(0, batch)

In [9]:
out =model({"meg" : batch}, segment_batch)

Crop & pad:  361


In [10]:
simpleconv

{'in_channels': {'meg': 270},
 'out_channels': 216,
 'hidden': {'meg': 320},
 'batch_norm': True,
 'depth': 4,
 'dilation_period': 1,
 'kernel_size': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 'skip': True,
 'subject_layers': False,
 'subject_dim': 0,
 'complex_out': False,
 'glu': 2,
 'glu_context': 1,
 'merger': False,
 'initial_linear': 270,
 'avg_pool_out': False,
 'adaptive_pooling_size': 1,
 'strides': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'padding': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'gelu': True,
 'merger_pos_dim': 2048,
 'auto_padding': False,
 'flatten_out': True,
 'conv_dropout': 0.2,
 'is_deformable_conv': False,
 'seq_len': 361,
 'n_subjects': 27,
 'flatten_out_channels': 264}

In [11]:
out.shape

torch.Size([256, 216, 361])

In [11]:
simpleconv={
      "in_channels": {"meg" : 207},
      "out_channels" :216,
      "hidden":
        {"meg": 320},
      "batch_norm": True,
      "depth": 4,
      "dilation_period": 1,
      "kernel_size": [3,3,3,3,3,3,3,3,3,3],
      "skip": True, 
      "subject_layers": False,
      "subject_dim": 0,
      "complex_out": False, 
      "glu": 2, 
      "glu_context": 1, 
      "merger": False,
      "initial_linear": 270,
      "avg_pool_out" : False,
      "adaptive_pooling_size" : 1,
      "strides" : [1,1,1,1,1,1,1,1,1,1],
      "padding": [1,1,1,1,1,1,1,1,1,1,1,1],
      "gelu": True,
      "merger_pos_dim": 2048,
      "auto_padding" : False,
      "flatten_out" : True,
       "conv_dropout" : 0.2,
     "is_deformable_conv": False,
       "seq_len" : 361, 
      "n_subjects" : 27,
      "flatten_out_channels" : 264}

In [12]:
cnn_transformer = CNNTransformer(in_channels_tranformer=264,out_channels_transformer=1024, dim_ff = 2048, 
                 nhead =8,
                 positional_embedding=False,
                 positional_embedding_dropout=0.0, **simpleconv)

264


In [13]:
cnn_transformer

CNNTransformer(
  (initial_linear): Sequential(
    (0): Conv1d(207, 270, kernel_size=(1,), stride=(1,))
  )
  (encoders): ModuleDict(
    (meg): ConvSequence(
      (sequence): ModuleList(
        (0): Sequential(
          (0): Conv1d(270, 320, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): GELU(approximate='none')
          (3): Dropout(p=0.2, inplace=False)
        )
        (1-2): 2 x Sequential(
          (0): Conv1d(320, 320, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): GELU(approximate='none')
          (3): Dropout(p=0.2, inplace=False)
        )
        (3): Sequential(
          (0): Conv1d(320, 264, kernel_size=(3,), stride=(1,), padding=(1,))
        )
      )
      (glus): ModuleList(
        (0): None
        (1): Sequential(
          (0): Conv1d(320, 64

In [14]:
batch.shape

torch.Size([256, 207, 362])

In [15]:
out =cnn_transformer({"meg" : batch}, segment_batch)

Crop & pad:  361
torch.Size([256, 264, 361])
torch.Size([256, 361, 264])


In [16]:
out.shape

torch.Size([256, 361, 1024])